<a href="https://colab.research.google.com/github/yh-hacker/svc-online/blob/main/mvsep_mdx23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MVSep-MDX23 Colab Fork v2.4
# 人声伴奏分离

In [ ]:
#@markdown #安装
#@markdown *Run this cell to install MVSep-MDX23*
print('Installing... This will take between 1 and 15 minutes, depending of how crappy Colab currently is...')
%cd /content

!git clone -b v2.4 https://github.com/jarredou/MVSEP-MDX23-Colab_v2  &> /dev/null
%cd /content/MVSEP-MDX23-Colab_v2
print('Installing dependencies...')
!pip install -r requirements.txt &> /dev/null
# onnxruntime-gpu nightly fix for cuda12.2
print('Installing onnxruntime CUDA 12.x fix...')
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/  &> /dev/null
print('Installation done !')

In [ ]:
#@title 上传.wav音频文件
import os
from google.colab import files

save_dir = "/content/input"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_path = os.path.join(save_dir, file_name)
with open(file_path, "wb") as f:
    f.write(uploaded[file_name])
print(f"File saved at: {file_path}")


In [ ]:
!rm -rf /content/output/*
!rm /content/output.zip

#@markdown #Separation
from pathlib import Path
import glob

%cd /content/MVSEP-MDX23-Colab_v2

#@markdown ---
#@markdown #### separation config:
input = '/content/input' #@param {type:"string"}
output_folder = '/content/output' #@param {type:"string"}

output_format = 'FLAC' #@param ["PCM_16", "FLOAT", "FLAC"]
Separation_mode = 'Vocals/Instrumental' #@param ["Vocals/Instrumental", "4-STEMS"]
input_gain = 0 #@param [0, -3, -6] {type:"raw"}
restore_gain_after_separation = False #@param {type:"boolean"}
filter_vocals_below_50hz = False #@param {type:"boolean"}
#@markdown ___
##@markdown

#@markdown  ### Model config:

#@markdown  *Set BigShifts=1 to disable that feature*
BigShifts = 3 #@param {type:"slider", min:1, max:41, step:1}
#@markdown ---
BSRoformer_model = 'ep_317_1297' #@param ["ep_317_1297", "ep_368_1296"]
weight_BSRoformer = 10 #@param {type:"slider", min:0, max:10, step:1}
##@markdown ---
weight_InstVoc = 4 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
use_VitLarge = True #@param {type:"boolean"}
weight_VitLarge = 1 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
use_InstHQ4 = False #@param {type:"boolean"}
weight_InstHQ4 = 2 #@param {type:"slider", min:0, max:10, step:1}
overlap_InstHQ4 = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
use_VOCFT = False #@param {type:"boolean"}
weight_VOCFT = 2 #@param {type:"slider", min:0, max:10, step:1}
overlap_VOCFT = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
#@markdown  *Demucs is only used in 4-STEMS mode.*
overlap_demucs = 0.6 #@param {type:"slider", min:0, max:0.95, step:0.05}

use_InstVoc_ = '--use_InstVoc' #forced use
use_BSRoformer_ =  '--use_BSRoformer' #forced use
use_VOCFT_ = '--use_VOCFT' if use_VOCFT is True else ''
use_VitLarge_ = '--use_VitLarge' if use_VitLarge is True else ''
use_InstHQ4_ = '--use_InstHQ4' if use_InstHQ4 is True else ''
restore_gain = '--restore_gain' if restore_gain_after_separation is True else ''
vocals_only = '--vocals_only' if Separation_mode == 'Vocals/Instrumental' else ''
filter_vocals = '--filter_vocals' if filter_vocals_below_50hz is True else ''

if Path(input).is_file():
  file_path = input
  Path(output_folder).mkdir(parents=True, exist_ok=True)
  !python inference.py \
        --input_audio "{file_path}" \
        --large_gpu \
        --BSRoformer_model {BSRoformer_model} \
        --weight_BSRoformer {weight_BSRoformer} \
        --weight_InstVoc {weight_InstVoc} \
        --weight_InstHQ4 {weight_InstHQ4} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_VitLarge {weight_VitLarge} \
        --overlap_demucs {overlap_demucs} \
        --overlap_VOCFT {overlap_VOCFT} \
        --overlap_InstHQ4 {overlap_InstHQ4} \
        --output_format {output_format} \
        --BigShifts {BigShifts} \
        --output_folder "{output_folder}" \
        --input_gain {input_gain} \
        {filter_vocals} \
        {restore_gain} \
        {vocals_only} \
        {use_VitLarge_} \
        {use_VOCFT_} \
        {use_InstHQ4_} \
        {use_InstVoc_} \
        {use_BSRoformer_}


else:
    file_paths = sorted(glob.glob(input + "/*"))[:]
    input_audio_args = ' '.join([f'"{path}"' for path in file_paths])
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    !python inference.py \
        --input_audio {input_audio_args} \
        --large_gpu \
        --BSRoformer_model {BSRoformer_model} \
        --weight_BSRoformer {weight_BSRoformer} \
        --weight_InstVoc {weight_InstVoc} \
        --weight_InstHQ4 {weight_InstHQ4} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_VitLarge {weight_VitLarge} \
        --overlap_demucs {overlap_demucs} \
        --overlap_VOCFT {overlap_VOCFT} \
        --overlap_InstHQ4 {overlap_InstHQ4} \
        --output_format {output_format} \
        --BigShifts {BigShifts} \
        --output_folder "{output_folder}" \
        --input_gain {input_gain} \
        {filter_vocals} \
        {restore_gain} \
        {vocals_only} \
        {use_VitLarge_} \
        {use_VOCFT_} \
        {use_InstHQ4_} \
        {use_InstVoc_} \
        {use_BSRoformer_}

download_after_inference = True  #@param {type:"boolean"}

if download_after_inference:
  %cd /content/
  !zip -r output.zip /content/output/
  from google.colab import files
  files.download("/content/output.zip")